In [37]:
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.utils.data as Data
import torch

In [38]:
# 手写数字训练数据集
mnist = np.load("../../../../Other/datasets/NumPy/mnist.npz")
X_train, y_train, X_test, y_test = torch.unsqueeze(torch.tensor(mnist['x_train']), 1).to(dtype=torch.float32), \
                                   torch.tensor(mnist['y_train']).to(dtype=torch.long), \
                                   torch.unsqueeze(torch.tensor(mnist['x_test']), 1).to(dtype=torch.float32), \
                                   torch.tensor(mnist['y_test']).to(dtype=torch.long)
X_train, X_test = X_train / 255.0, X_test / 255.0

In [39]:
batch_size = 200
train_data = Data.TensorDataset(X_train, y_train)
train_loader = Data.DataLoader(dataset=train_data,
                               batch_size=batch_size,
                               shuffle=True)
train_X = Data.DataLoader(X_train, batch_size=batch_size)
test_X = Data.DataLoader(X_test, batch_size=batch_size)

In [40]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.featrues = nn.Sequential(  # 内部实现了forward函数;各模块顺序执行
            nn.Conv2d(1, 6, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(400, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),  # 10分类问题
        )

    def forward(self, x):
        x = self.featrues(x)
        x = x.reshape(x.size()[0], -1)
        x = self.classifier(x)
        return x

In [41]:
learning_rate = 0.01
criteon = nn.CrossEntropyLoss()  # 定义损失函数为交叉熵
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = LeNet()
lenet = net.to(device)  # 模型设备转移必须在优化器定义前执行

# RMSprop优化器
optimizer = optim.RMSprop(net.parameters(),
                          lr=learning_rate,  # 学习率
                          momentum=0.4,  # 动量参数\alpha;默认momentum=0
                          alpha=0.9,  # 衰减速率\rho
                          eps=1e-10)  # 分母中的小常数\delta

In [42]:
from torch_model.train_evaluate import Train_Evaluate

t_and_v = Train_Evaluate(model=lenet, optimizer=optimizer, criterion=criteon, epochs=5)

In [43]:
metric_reslut = t_and_v.train_eval(train_loader=train_loader, valid_sets=[(train_X, y_train), (test_X, y_test)])
metric_reslut  # 每一轮验证数据集的损失值

Train Epoch: 0  [0    /60000 (0  %)]	Loss: 2.304123
Train Epoch: 0  [2000 /60000 (3  %)]	Loss: 2.280261
Train Epoch: 0  [4000 /60000 (7  %)]	Loss: 1.824708
Train Epoch: 0  [6000 /60000 (10 %)]	Loss: 0.865382
Train Epoch: 0  [8000 /60000 (13 %)]	Loss: 0.500940
Train Epoch: 0  [10000/60000 (17 %)]	Loss: 0.312089
Train Epoch: 0  [12000/60000 (20 %)]	Loss: 0.306018
Train Epoch: 0  [14000/60000 (23 %)]	Loss: 0.241783
Train Epoch: 0  [16000/60000 (27 %)]	Loss: 0.155759
Train Epoch: 0  [18000/60000 (30 %)]	Loss: 0.216600
Train Epoch: 0  [20000/60000 (33 %)]	Loss: 0.151453
Train Epoch: 0  [22000/60000 (37 %)]	Loss: 0.231087
Train Epoch: 0  [24000/60000 (40 %)]	Loss: 0.257188
Train Epoch: 0  [26000/60000 (43 %)]	Loss: 0.126243
Train Epoch: 0  [28000/60000 (47 %)]	Loss: 0.216057
Train Epoch: 0  [30000/60000 (50 %)]	Loss: 0.119573
Train Epoch: 0  [32000/60000 (53 %)]	Loss: 0.098021
Train Epoch: 0  [34000/60000 (57 %)]	Loss: 0.145877
Train Epoch: 0  [36000/60000 (60 %)]	Loss: 0.078281
Train Epoch:

[[0.08570770174264908, 0.08818972110748291],
 [0.08514104783535004, 0.0923006534576416],
 [0.09313888102769852, 0.10468745231628418],
 [0.07586648315191269, 0.09525709599256516],
 [0.07131423056125641, 0.09882934391498566]]